In [1]:
import cudf

data=cudf.read_csv("/home/pavit21178/Nalin_OFF/Data/OFF_FNDDS_COLS_NOVA.csv",index_col=0)
# keep only data w 1,2,3,4 in the novaclass column
data = data[data['novaclass'].isin([1,2,3,4])]







In [2]:
# use random forest to predict the NOVA group
from cuml.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from cuml.metrics import accuracy_score

X = data.drop('novaclass', axis=1)
y=data['novaclass']



In [3]:
from cuml.preprocessing import SimpleImputer
import cupy as cp
imp_mean = SimpleImputer(missing_values=cp.nan, strategy='mean')


In [4]:
from cuml.preprocessing import StandardScaler
from cuml.preprocessing import SimpleImputer
import cupy as cp
imp_mean = SimpleImputer(missing_values=cp.nan, strategy='mean')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train=imp_mean.fit_transform(X_train)
X_test=imp_mean.transform(X_test)




In [12]:
from lightgbm import LGBMClassifier
X_train_np=X_train.to_pandas()
X_test_np=X_test.to_pandas()
y_train_np=y_train.to_pandas()
y_test_np=y_test.to_pandas()
lgbm=LGBMClassifier(verbose=-1,device='gpu')
lgbm.fit(X_train_np,y_train_np)
y_pred=lgbm.predict(X_test_np)


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


In [13]:
print(accuracy_score(y_test,y_pred))
y_test_np=y_test.to_numpy()
# y_pred_np=y_pred.to_numpy()
from sklearn.metrics import classification_report
print(classification_report(y_test_np, y_pred))

0.7624432444572449
              precision    recall  f1-score   support

         1.0       0.78      0.62      0.69     20092
         2.0       0.88      0.57      0.70     11288
         3.0       0.70      0.38      0.49     34605
         4.0       0.76      0.93      0.84    109030

    accuracy                           0.76    175015
   macro avg       0.78      0.63      0.68    175015
weighted avg       0.76      0.76      0.74    175015



In [14]:
param_grid = {
    'boosting_type': ['gbdt', 'dart'],           # Boosting type: Gradient Boosting Decision Tree (GBDT) or Dropouts meet Multiple Additive Regression Trees (DART)
    'num_leaves': [31, 63, 127],                 # Number of leaves in one tree, higher values can increase accuracy but also increase risk of overfitting
    'max_depth': [-1, 10, 20, 30],               # Maximum tree depth for base learners, -1 means no limit
    'learning_rate': [0.01, 0.05, 0.1],          # Learning rate: smaller values lead to better accuracy but require more training rounds
    'n_estimators': [100, 500, 1000,2000,3000,4000,5000,10000,20000],            # Number of boosted trees to fit
    'subsample': [0.6, 0.8, 1.0],                # Fraction of samples used for fitting individual base learners
    'colsample_bytree': [0.6, 0.8, 1.0],         # Fraction of features used for fitting individual base learners
    'reg_alpha': [0, 0.1, 1],                    # L1 regularization term on weights
    'reg_lambda': [0, 0.1, 1],                   # L2 regularization term on weights
    'min_split_gain': [0, 0.01, 0.1],            # Minimum loss reduction required to make a further partition on a leaf node of the tree
    'min_child_weight': [1, 5, 10],              # Minimum sum of instance weight (hessian) needed in a child (leaf)
    'scale_pos_weight': [1, 10, 20]              # Balancing of positive and negative weights
}


In [15]:
from sklearn.model_selection import RandomizedSearchCV

# Set up the RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=lgbm, param_distributions=param_grid, n_iter=10, cv=5, random_state=42, verbose=2)

# Fit the RandomizedSearchCV on the imputed training data
random_search.fit(X_train_np, y_train_np)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END boosting_type=gbdt, colsample_bytree=0.8, learning_rate=0.05, max_depth=20, min_child_weight=5, min_split_gain=0.1, n_estimators=100, num_leaves=127, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=20, subsample=1.0; total time=   9.2s
[CV] END boosting_type=gbdt, colsample_bytree=0.8, learning_rate=0.05, max_depth=20, min_child_weight=5, min_split_gain=0.1, n_estimators=100, num_leaves=127, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=20, subsample=1.0; total time=   9.1s
[CV] END boosting_type=gbdt, colsample_bytree=0.8, learning_rate=0.05, max_depth=20, min_child_weight=5, min_split_gain=0.1, n_estimators=100, num_leaves=127, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=20, subsample=1.0; total time=   9.1s
[CV] END boosting_type=gbdt, colsample_bytree=0.8, learning_rate=0.05, max_depth=20, min_child_weight=5, min_split_gain=0.1, n_estimators=100, num_leaves=127, reg_alpha=0.1, reg_lambda=1, scale_pos_weight=20, s

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(device='gpu', verbose=-1),
                   param_distributions={'boosting_type': ['gbdt', 'dart'],
                                        'colsample_bytree': [0.6, 0.8, 1.0],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [-1, 10, 20, 30],
                                        'min_child_weight': [1, 5, 10],
                                        'min_split_gain': [0, 0.01, 0.1],
                                        'n_estimators': [100, 500, 1000],
                                        'num_leaves': [31, 63, 127],
                                        'reg_alpha': [0, 0.1, 1],
                                        'reg_lambda': [0, 0.1, 1],
                                        'scale_pos_weight': [1, 10, 20],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, verbose=2)

In [18]:
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation score: ", random_search.best_score_)

# Evaluate the best model on the imputed test set
best_model = random_search.best_estimator_
test_score = best_model.score(X_test_np, y_test_np)
print("Test set score of the best model: ", test_score)

Best parameters found:  {'subsample': 0.8, 'scale_pos_weight': 1, 'reg_lambda': 0.1, 'reg_alpha': 1, 'num_leaves': 127, 'n_estimators': 1000, 'min_split_gain': 0, 'min_child_weight': 1, 'max_depth': -1, 'learning_rate': 0.1, 'colsample_bytree': 0.8, 'boosting_type': 'gbdt'}
Best cross-validation score:  0.8057180813073165
Test set score of the best model:  0.8078507556495158


In [19]:
lgbm=LGBMClassifier(verbose=-1,device='gpu',**random_search.best_params_)

In [51]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    X_train = X_train.to_pandas()
    y_train = y_train.to_pandas()
    X_test = X_test.to_pandas()
    y_test = y_test.to_pandas()

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        train_indices = cp.asnumpy(train_indices)
        val_indices = cp.asnumpy(val_indices)

        X_fold_train, X_fold_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_fold_train, y_fold_val = y_train.iloc[train_indices], y_train.iloc[val_indices]

        smote = SMOTE()
        X_fold_train_resampled, y_fold_train_resampled = smote.fit_resample(X_fold_train, y_fold_train)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)

        y_train_pred = classifier.predict(X_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled, y_train_pred)
        train_scores.append(train_score)

        fold_accuracy_train = accuracy_score(y_fold_train_resampled, y_train_pred)
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled, y_train_pred))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        y_val_pred = classifier.predict(X_fold_val)
        val_score = accuracy_score(y_fold_val, y_val_pred)
        val_scores.append(val_score)

        fold_accuracy_val = accuracy_score(y_fold_val, y_val_pred)
        accuracies_val.append(fold_accuracy_val)

        val_mcc.append(matthews_corrcoef(y_fold_val, y_val_pred))
        val_f1.append(precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test, y_pred)
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))



In [52]:
evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, lgbm, num_folds=2, save_path="/home/pavit21178/Nalin_OFF/Models_44/LGBM", model_name="LGBM_strat_Kfold_Smote")

Average Accuracy Val K-Fold: 75.5174
Average Accuracy Train K-Fold: 88.8461
Average MCC Val K-Fold: 0.5807
Average MCC Train K-Fold: 0.8548
Average F1 Score Val K-Fold: 0.7606
Average F1 Score Train K-Fold: 0.8895
Average Precision Val K-Fold: 0.7806
Average Precision Train K-Fold: 0.9001
Average Recall Val K-Fold: 0.7552
Average Recall Train K-Fold: 0.8885
Accuracy Test: 75.36%
MCC Test: 0.5776
F1 Score Test: 0.7585
Precision Test: 0.7781
Recall Test: 0.7536
Classification Report:
              precision    recall  f1-score   support

         1.0       0.49      0.84      0.62     20092
         2.0       0.83      0.62      0.71     11288
         3.0       0.66      0.58      0.62     34605
         4.0       0.86      0.81      0.83    109030

    accuracy                           0.75    175015
   macro avg       0.71      0.71      0.70    175015
weighted avg       0.78      0.75      0.76    175015

Confusion Matrix Test:
[[16895   130   957  2110]
 [ 3519  6955    94   720]
 

In [53]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    X_train = X_train.to_pandas()
    y_train = y_train.to_pandas()
    X_test = X_test.to_pandas()
    y_test = y_test.to_pandas()

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        train_indices = cp.asnumpy(train_indices)
        val_indices = cp.asnumpy(val_indices)

        X_fold_train, X_fold_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_fold_train, y_fold_val = y_train.iloc[train_indices], y_train.iloc[val_indices]

        smote = SMOTE()
        X_fold_train_resampled, y_fold_train_resampled = smote.fit_resample(X_fold_train, y_fold_train)

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)

        y_train_pred = classifier.predict(X_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled, y_train_pred)
        train_scores.append(train_score)

        fold_accuracy_train = accuracy_score(y_fold_train_resampled, y_train_pred)
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled, y_train_pred))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        y_val_pred = classifier.predict(X_fold_val)
        val_score = accuracy_score(y_fold_val, y_val_pred)
        val_scores.append(val_score)

        fold_accuracy_val = accuracy_score(y_fold_val, y_val_pred)
        accuracies_val.append(fold_accuracy_val)

        val_mcc.append(matthews_corrcoef(y_fold_val, y_val_pred))
        val_f1.append(precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test, y_pred)
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))



In [54]:
evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, lgbm, num_folds=2, save_path="/home/pavit21178/Nalin_OFF/Models_44/LGBM", model_name="LGBM_Kfold_Smote")

Average Accuracy Val K-Fold: 75.6085
Average Accuracy Train K-Fold: 89.0428
Average MCC Val K-Fold: 0.5831
Average MCC Train K-Fold: 0.8575
Average F1 Score Val K-Fold: 0.7621
Average F1 Score Train K-Fold: 0.8910
Average Precision Val K-Fold: 0.7822
Average Precision Train K-Fold: 0.9014
Average Recall Val K-Fold: 0.7561
Average Recall Train K-Fold: 0.8904
Accuracy Test: 75.59%
MCC Test: 0.5838
F1 Score Test: 0.7621
Precision Test: 0.7827
Recall Test: 0.7559
Classification Report:
              precision    recall  f1-score   support

         1.0       0.76      0.68      0.72     20092
         2.0       0.42      0.91      0.58     11288
         3.0       0.66      0.58      0.62     34605
         4.0       0.86      0.81      0.84    109030

    accuracy                           0.76    175015
   macro avg       0.68      0.75      0.69    175015
weighted avg       0.78      0.76      0.76    175015

Confusion Matrix Test:
[[13699  3278  1010  2105]
 [  162 10309    92   725]
 

In [55]:
import numpy as np
import pandas as pd
from cuml.ensemble import RandomForestClassifier as cuRF
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, matthews_corrcoef, precision_recall_fscore_support,
    classification_report, confusion_matrix
)
from imblearn.over_sampling import SMOTE
import os
import joblib
import cudf
import cupy as cp

def evaluate_classifier_with_stratified(X_train, y_train, X_test, y_test, classifier, num_folds=10, save_path=None, model_name=None):
    k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

    accuracies_val = []
    accuracies_train = []
    train_scores = []
    val_scores = []
    train_mcc = []
    val_mcc = []
    train_f1 = []
    val_f1 = []
    train_precision = []
    val_precision = []
    train_recall = []
    val_recall = []

    X_train = X_train.to_pandas()
    y_train = y_train.to_pandas()
    X_test = X_test.to_pandas()
    y_test = y_test.to_pandas()

    for fold_idx, (train_indices, val_indices) in enumerate(k_fold.split(X_train, y_train), 1):
        train_indices = cp.asnumpy(train_indices)
        val_indices = cp.asnumpy(val_indices)

        X_fold_train, X_fold_val = X_train.iloc[train_indices], X_train.iloc[val_indices]
        y_fold_train, y_fold_val = y_train.iloc[train_indices], y_train.iloc[val_indices]

        X_fold_train_resampled, y_fold_train_resampled = X_fold_train, y_fold_train

        classifier.fit(X_fold_train_resampled, y_fold_train_resampled)

        y_train_pred = classifier.predict(X_fold_train_resampled)
        train_score = accuracy_score(y_fold_train_resampled, y_train_pred)
        train_scores.append(train_score)

        fold_accuracy_train = accuracy_score(y_fold_train_resampled, y_train_pred)
        accuracies_train.append(fold_accuracy_train)

        train_mcc.append(matthews_corrcoef(y_fold_train_resampled, y_train_pred))
        train_f1.append(precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_train_resampled, y_train_pred, average='weighted')
        train_precision.append(precision)
        train_recall.append(recall)

        y_val_pred = classifier.predict(X_fold_val)
        val_score = accuracy_score(y_fold_val, y_val_pred)
        val_scores.append(val_score)

        fold_accuracy_val = accuracy_score(y_fold_val, y_val_pred)
        accuracies_val.append(fold_accuracy_val)

        val_mcc.append(matthews_corrcoef(y_fold_val, y_val_pred))
        val_f1.append(precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')[2])
        precision, recall, _, _ = precision_recall_fscore_support(y_fold_val, y_val_pred, average='weighted')
        val_precision.append(precision)
        val_recall.append(recall)

    average_accuracy_val = sum(accuracies_val) / num_folds
    print(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}')

    average_accuracy_train = sum(accuracies_train) / num_folds
    print(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}')

    average_mcc_val = sum(val_mcc) / num_folds
    print(f'Average MCC Val K-Fold: {average_mcc_val:.4f}')

    average_mcc_train = sum(train_mcc) / num_folds
    print(f'Average MCC Train K-Fold: {average_mcc_train:.4f}')

    average_f1_val = sum(val_f1) / num_folds
    print(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}')

    average_f1_train = sum(train_f1) / num_folds
    print(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}')

    average_precision_val = sum(val_precision) / num_folds
    print(f'Average Precision Val K-Fold: {average_precision_val:.4f}')

    average_precision_train = sum(train_precision) / num_folds
    print(f'Average Precision Train K-Fold: {average_precision_train:.4f}')

    average_recall_val = sum(val_recall) / num_folds
    print(f'Average Recall Val K-Fold: {average_recall_val:.4f}')

    average_recall_train = sum(train_recall) / num_folds
    print(f'Average Recall Train K-Fold: {average_recall_train:.4f}')

    y_pred = classifier.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy * 100:.2f}%')

    test_mcc = matthews_corrcoef(y_test, y_pred)
    print(f'MCC Test: {test_mcc:.4f}')

    test_f1 = precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]
    print(f'F1 Score Test: {test_f1:.4f}')

    test_precision, test_recall, _, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    print(f'Precision Test: {test_precision:.4f}')
    print(f'Recall Test: {test_recall:.4f}')

    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)

    conf_matrix_test = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix Test:')
    print(conf_matrix_test)

    if save_path and model_name:
        output_filename = os.path.join(save_path, f"{model_name}_outputs.txt")
        with open(output_filename, 'w') as f:
            f.write(f'Average Accuracy Val K-Fold: {average_accuracy_val * 100:.4f}\n')
            f.write(f'Average Accuracy Train K-Fold: {average_accuracy_train * 100:.4f}\n')
            f.write(f'Average MCC Val K-Fold: {average_mcc_val:.4f}\n')
            f.write(f'Average MCC Train K-Fold: {average_mcc_train:.4f}\n')
            f.write(f'Average F1 Score Val K-Fold: {average_f1_val:.4f}\n')
            f.write(f'Average F1 Score Train K-Fold: {average_f1_train:.4f}\n')
            f.write(f'Average Precision Val K-Fold: {average_precision_val:.4f}\n')
            f.write(f'Average Precision Train K-Fold: {average_precision_train:.4f}\n')
            f.write(f'Average Recall Val K-Fold: {average_recall_val:.4f}\n')
            f.write(f'Average Recall Train K-Fold: {average_recall_train:.4f}\n')
            f.write(f'Accuracy Test: {accuracy * 100:.2f}%\n')
            f.write(f'MCC Test: {test_mcc:.4f}\n')
            f.write(f'F1 Score Test: {test_f1:.4f}\n')
            f.write(f'Precision Test: {test_precision:.4f}\n')
            f.write(f'Recall Test: {test_recall:.4f}\n')
            f.write('Classification Report:\n')
            f.write(class_report + '\n')
            f.write('Confusion Matrix Test:\n')
            f.write(str(conf_matrix_test) + '\n')
        print(f"Outputs saved to: {output_filename}")
        joblib.dump(classifier, os.path.join(save_path, f"{model_name}.joblib"))



In [57]:
evaluate_classifier_with_stratified(X_train, y_train, X_test, y_test, lgbm, num_folds=2, save_path="/home/pavit21178/Nalin_OFF/Models_44/LGBM", model_name="LGBM_strat")

Average Accuracy Val K-Fold: 80.0978
Average Accuracy Train K-Fold: 89.1759
Average MCC Val K-Fold: 0.6200
Average MCC Train K-Fold: 0.7995
Average F1 Score Val K-Fold: 0.7912
Average F1 Score Train K-Fold: 0.8883
Average Precision Val K-Fold: 0.7998
Average Precision Train K-Fold: 0.8964
Average Recall Val K-Fold: 0.8010
Average Recall Train K-Fold: 0.8918
Accuracy Test: 80.01%
MCC Test: 0.6190
F1 Score Test: 0.7902
Precision Test: 0.7988
Recall Test: 0.8001
Classification Report:
              precision    recall  f1-score   support

         1.0       0.82      0.66      0.73     20092
         2.0       0.89      0.61      0.72     11288
         3.0       0.74      0.53      0.62     34605
         4.0       0.80      0.93      0.86    109030

    accuracy                           0.80    175015
   macro avg       0.81      0.68      0.73    175015
weighted avg       0.80      0.80      0.79    175015

Confusion Matrix Test:
[[ 13305    116    934   5737]
 [   122   6846     83  